<a href="https://colab.research.google.com/github/Mye0n9/AAI_Pitcher/blob/CNN_Testing/Pitcher_CNN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import glob
import cv2
import re
import random
import json
import os
from google.colab.patches import cv2_imshow


print("NumPy Version :{}".format(np.__version__))
print("TensorFlow Version :{}".format(tf.__version__))
print("Matplotlib Version :{}".format(plt.matplotlib.__version__))

NumPy Version :1.22.4
TensorFlow Version :2.12.0
Matplotlib Version :3.7.1


Reading Data (Pickle Data)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
x_Data = []
y_Data = []
for i in range(120):
  print(i)
  if i not in [96, 136, 146, 197, 220, 223]:
    f_path = '/content/drive/MyDrive/PKL_PITCHER/img_merged_{num}.pkl'.format(num = i)
    df = pd.read_pickle(f_path)
    for j in range(len(df)):
      x_Data.append(df['wrist'][j])
      y_Data.append(0.0)
      x_Data.append(df['elbow'][j])
      y_Data.append(1.0)
      x_Data.append(df['shoulder'][j])
      y_Data.append(2.0)

x_Data = np.array(x_Data)
y_Data = np.array(y_Data)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119


Train/Test split

In [7]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_Data, y_Data, test_size=0.2, random_state=42)

In [ ]:
# x_train = np.array(x_train)
# x_test = np.array(x_test)
# y_train = np.array(y_train)
# y_test = np.array(y_test)

Model

In [4]:
IMG_SIZE = 32
n_class = 3
model = tf.keras.models.Sequential()
# conv 1
model.add(tf.keras.layers.Conv2D(64,3,padding='same',activation='relu',input_shape=(IMG_SIZE,IMG_SIZE,3)))
model.add(tf.keras.layers.Dropout(rate=0.5))                          #  DO1
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
# conv 2
model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.Dropout(rate=0.5))                          # DO2
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
# conv 3
model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.Dropout(rate=0.5))                           # DO3
model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# dense layers
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(n_class, activation='softmax')) 

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 32, 32, 64)        1792      
                                                                 
 dropout_3 (Dropout)         (None, 32, 32, 64)        0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 16, 16, 64)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 16, 16, 64)        36928     
                                                                 
 dropout_4 (Dropout)         (None, 16, 16, 64)        0         
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 8, 8, 64)         0         
 2D)                                                  

In [8]:
model.compile(optimizer='SGD',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
saved_model = '../files/pitcher_cnn3_model.h5'
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20,restore_best_weights=True)
mc = tf.keras.callbacks.ModelCheckpoint(saved_model, monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

In [10]:
history = model.fit(x_train, y_train, epochs=100,
                      callbacks=[es,mc],
                      validation_data=(x_test, y_test)
                 )

Epoch 1/100
981/981 [==============================] - ETA: 0s - loss: 1.0988 - accuracy: 0.3334
Epoch 1: val_accuracy improved from -inf to 0.32879, saving model to ../files/pitcher_cnn3_model.h5
981/981 [==============================] - 16s 7ms/step - loss: 1.0988 - accuracy: 0.3334 - val_loss: 1.0987 - val_accuracy: 0.3288
Epoch 2/100
975/981 [============================>.] - ETA: 0s - loss: 1.0987 - accuracy: 0.3371
Epoch 2: val_accuracy improved from 0.32879 to 0.33312, saving model to ../files/pitcher_cnn3_model.h5
981/981 [==============================] - 5s 5ms/step - loss: 1.0987 - accuracy: 0.3369 - val_loss: 1.0987 - val_accuracy: 0.3331
Epoch 3/100
974/981 [============================>.] - ETA: 0s - loss: 1.0988 - accuracy: 0.3308
Epoch 3: val_accuracy did not improve from 0.33312
981/981 [==============================] - 6s 6ms/step - loss: 1.0988 - accuracy: 0.3310 - val_loss: 1.0987 - val_accuracy: 0.3331
Epoch 4/100
971/981 [============================>.] - ETA: 0